In [1]:
# Import necessary libraries
import os                       # Operating system interface for file operations
import numpy as np              # Numerical computing library
from PIL import Image           # Python Imaging Library for image processing
from tqdm import tqdm           # Progress bar utility for loops
from sklearn.model_selection import train_test_split  # For splitting data into train/test sets

# Import TensorFlow and Keras components
import tensorflow as tf         # Machine learning framework
from tensorflow.keras.models import Sequential  # For creating sequential neural network models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense  # Neural network layer types
from tensorflow.keras.optimizers import Adam    # Optimization algorithm for training

In [2]:
# Define the save directory for models, data, and logs
save_dir = "./fashion_ai_model_data"  # Set this to a folder on your local machine
os.makedirs(save_dir, exist_ok=True)

# Path to your dataset
image_dir = "DeepFashionData/images"  # Change this to your actual image folder path
shape_labels_path = "DeepFashionData/labels/shape/shape_anno_all.txt"  # Change this to your shape labels file path

In [3]:
# Load shape labels
shape_labels = {}
with open(shape_labels_path) as f:
    for line in f:
        parts = line.strip().split()
        img_name = parts[0]
        shape_labels[img_name] = list(map(int, parts[1:]))

# Define the target size for resizing images
target_size = (224, 224)

In [4]:
# Load and preprocess images (using 10% of the data for testing purposes)
X = []  # Images
y = []  # Shape labels

In [ ]:
sample_size = int(len(shape_labels) * 1)  # Use 10% of the dataset for quick testing

for img_name, shape in tqdm(list(shape_labels.items())[:sample_size]):
    img_path = os.path.join(image_dir, img_name)
    try:
        img = Image.open(img_path).convert("RGB")
        img = img.resize(target_size)  # Resize image to 224x224
        X.append(np.array(img) / 255.0)  # Normalize image to [0, 1]
        y.append(shape)
    except Exception as e:
        print(f"Error loading 2 {img_name}: {e}")
        continue  # Skip unreadable images

X = np.array(X)
y = np.array(y)

 83%|████████▎ | 35124/42544 [12:54<02:45, 44.85it/s]

In [ ]:
# Save processed data (images and labels)
np.save(os.path.join(save_dir, 'X.npy'), X)
np.save(os.path.join(save_dir, 'y.npy'), y)
print(f"Processed data saved to {save_dir}")

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save training and testing data
np.save(os.path.join(save_dir, 'X_train.npy'), X_train)
np.save(os.path.join(save_dir, 'X_test.npy'), X_test)
np.save(os.path.join(save_dir, 'y_train.npy'), y_train)
np.save(os.path.join(save_dir, 'y_test.npy'), y_test)
print(f"Train and test data saved to {save_dir}")

In [ ]:
# Assistant
# Define the CNN model
model = Sequential([
    # First convolutional layer with 32 filters of size 3x3, ReLU activation
    # Input shape expects images of size 224x224 with 3 color channels (RGB)
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    # Max pooling layer to reduce spatial dimensions by half
    MaxPooling2D(2, 2),
    # Second convolutional layer with 64 filters of size 3x3, ReLU activation
    Conv2D(64, (3, 3), activation='relu'),
    # Another max pooling layer to further reduce dimensions
    MaxPooling2D(2, 2),
    # Flatten the 2D feature maps to 1D feature vectors
    Flatten(),
    # Fully connected layer with 128 neurons and ReLU activation
    Dense(128, activation='relu'),
    # Output layer with neurons matching the number of classes
    # Softmax activation for multi-class classification probability distribution
    Dense(len(y[0]), activation='softmax')  # Adjust the output layer based on the number of labels
])

In [ ]:
optimizer = Adam(learning_rate=0.0001)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Save the model after every few epochs
epochs = 10
for epoch in range(epochs):
    print(f"Training epoch {epoch+1}/{epochs}")
    model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_test, y_test))

    # Save the model after each epoch
    model.save(os.path.join(save_dir, f"fashion_ai_model_epoch_{epoch+1}.keras"))
    print(f"Model saved after epoch {epoch+1}")
